In [1]:
import logging
import json
import os
import shutil
import requests
import pandas as pd
import duckdb
from datetime import datetime

# Use a logger instead of simple print statements for better compatibility with orchestrators and logging services (like AWS CloudWatch)
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

API_ROOT = "https://api.frankfurter.app/"

#TODO: Move "CURRENCIES" and "TO_CURRENCY" to a separate YAML file. That would allow to update the values without changing the code.
CURRENCIES = {
	"AUD": "Australian Dollar",
	"BGN": "Bulgarian Lev",
	"BRL": "Brazilian Real",
	"CAD": "Canadian Dollar",
	"CHF": "Swiss Franc",
	"CNY": "Chinese Renminbi Yuan",
	"CZK": "Czech Koruna",
	"DKK": "Danish Krone",
	"EUR": "Euro",
	"GBP": "British Pound",
	"HKD": "Hong Kong Dollar",
	"HUF": "Hungarian Forint",
	"IDR": "Indonesian Rupiah",
	"ILS": "Israeli New Sheqel",
	"INR": "Indian Rupee",
	"ISK": "Icelandic Króna",
	"JPY": "Japanese Yen",
	"KRW": "South Korean Won",
	"MXN": "Mexican Peso",
	"MYR": "Malaysian Ringgit",
	"NOK": "Norwegian Krone",
	"NZD": "New Zealand Dollar",
	"PHP": "Philippine Peso",
	"PLN": "Polish Złoty",
	"RON": "Romanian Leu",
	"SEK": "Swedish Krona",
	"SGD": "Singapore Dollar",
	"THB": "Thai Baht",
	"TRY": "Turkish Lira",
	"USD": "United States Dollar",
    "ZAR": "South African Rand"
}

TO_CURRENCY = "EUR"
# Remove the target currency because it cannot be converted to itself
FROM_CURRENCIES = [c for c in CURRENCIES.keys() if c != TO_CURRENCY]

def get_file_paths(directory):
    # get full path and basename for all files in directory
    files = []
    for entry in os.scandir(f'{directory}/'):
        if entry.is_file():
            files.append({"path": entry.path, "name": os.path.basename(entry.path)})
        else:
            # if the entry is a directory, recursively call get_file_paths
            files.extend(get_file_paths(entry.path))
    return files

def get_exchange_rates(api_root:str, date:str, from_currency:str, to_currency:str):
	''' 
	Returns the exchange rate between two currencies on a given date
	'''
	url = api_root + date + "?from=" + from_currency + "&to=" + to_currency
	r = requests.get(url)
	log.info(f'Requested url: {url} with HTTP status code {r.status_code}')

	#Extract and transform the data
	data = r.json()
	to_currency, to_amount = list(data['rates'].items())[0]
	output_data = {
		"date": data["date"],
		"from_currency": data["base"],
		"amount": data["amount"],
		"to_currency": to_currency,
		"exchange_rate": to_amount
	}
	return json.dumps(output_data)

def store_exchange_rates(api_response:str, path:str):
	''' 
	Takes the API response and stores the exchange rate between two currencies on a given date
	'''
	r = json.loads(api_response)
	ds = r["date"]
	from_currency = r["from_currency"]
	to_currency = r["to_currency"]
	
	#Use Pandas to convert to the required file format
	df = pd.json_normalize(r)
	#Add an ETL timestamp to know when the data was processed
	df = df.assign(updated_at=datetime.now())
	# I would use the parquet file format, but for simplicity's sake I'm using csv
	output_path = f'{path}/{from_currency}_{to_currency}_{ds}.csv'
	df.to_csv(output_path, index=False)

	log.info(f'Stored exchange rates for {from_currency} to {to_currency} on {ds} in {output_path}')

In [2]:
# This notebook cell performs a one-time export for the exchange rates for all the dates
dates = ['2024-02-12', '2024-02-13']

data_path = '../data'
rates_path = f'{data_path}/exchange_rates'

for ds in dates:
    # Create or replace a folder named after the date. Using the date allows for partitioning the data in
    # the future for improved performance (following Apache Hive folder structure and enabling the ability to push predicates down).
    dir = f'{rates_path}/{ds}'
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)
    for currency in FROM_CURRENCIES:
        try:
            r = get_exchange_rates(API_ROOT, ds, currency, TO_CURRENCY)
            store_exchange_rates(r, dir)
        except Exception as e:
            log.error(e)

INFO:__main__:Requested url: https://api.frankfurter.app/2024-02-12?from=AUD&to=EUR with HTTP status code 200
INFO:__main__:Stored exchange rates for AUD to EUR on 2024-02-12 in ../data/exchange_rates/2024-02-12/AUD_EUR_2024-02-12.csv
INFO:__main__:Requested url: https://api.frankfurter.app/2024-02-12?from=BGN&to=EUR with HTTP status code 200
INFO:__main__:Stored exchange rates for BGN to EUR on 2024-02-12 in ../data/exchange_rates/2024-02-12/BGN_EUR_2024-02-12.csv
INFO:__main__:Requested url: https://api.frankfurter.app/2024-02-12?from=BRL&to=EUR with HTTP status code 200
INFO:__main__:Stored exchange rates for BRL to EUR on 2024-02-12 in ../data/exchange_rates/2024-02-12/BRL_EUR_2024-02-12.csv
INFO:__main__:Requested url: https://api.frankfurter.app/2024-02-12?from=CAD&to=EUR with HTTP status code 200
INFO:__main__:Stored exchange rates for CAD to EUR on 2024-02-12 in ../data/exchange_rates/2024-02-12/CAD_EUR_2024-02-12.csv
INFO:__main__:Requested url: https://api.frankfurter.app/202

In [3]:
# I'm going full "scripting" mode starting here; the following transformations could be refactored into function or classes with more time.

# Get ecommerce data
STORE_ENDPOINT = "https://fakestoreapi.com/products/category/women%27s%20clothing"
r = requests.get(STORE_ENDPOINT).json()
store_items = pd.DataFrame(r)

#Explode "rating" (dict to columns)
store_items = store_items.join(pd.DataFrame(store_items["rating"].tolist())[["rate", "count"]])
store_items.drop("rating", axis=1, inplace=True)

store_items.rename(columns={"price": "price_usd", "rate": "rating_rate", "count": "rating_count"}, inplace=True)

store_items.head()

,id,title,price_usd,description,category,image,rating_rate,rating_count
0,15,BIYLACLESEN Women's 3-in-1 Snowboard Jacket Wi...,56.99,"Note:The Jackets is US standard size, Please c...",women's clothing,https://fakestoreapi.com/img/51Y5NI-I5jL._AC_U...,2.6,235
1,16,Lock and Love Women's Removable Hooded Faux Le...,29.95,100% POLYURETHANE(shell) 100% POLYESTER(lining...,women's clothing,https://fakestoreapi.com/img/81XH0e8fefL._AC_U...,2.9,340
2,17,Rain Jacket Women Windbreaker Striped Climbing...,39.99,Lightweight perfet for trip or casual wear---L...,women's clothing,https://fakestoreapi.com/img/71HblAHs5xL._AC_U...,3.8,679
3,18,MBJ Women's Solid Short Sleeve Boat Neck V,9.85,"95% RAYON 5% SPANDEX, Made in USA or Imported,...",women's clothing,https://fakestoreapi.com/img/71z3kpMAYsL._AC_U...,4.7,130
4,19,Opna Women's Short Sleeve Moisture,7.95,"100% Polyester, Machine wash, 100% cationic po...",women's clothing,https://fakestoreapi.com/img/51eg55uWmdL._AC_U...,4.5,146


In [4]:
# Pull the exchange rates from stored files
rate_files = get_file_paths(rates_path)

# I'm pulling all the rates datasets, but in a real scenario I would only pull the required dates and currencies (for example, using AWS S3 and 
# the AWS Athena query engine over parquet files)
rates_df = pd.concat((pd.read_csv(f["path"]) for f in rate_files), ignore_index=True)

# Only keep USD rates for the required dates
rates_df = rates_df[(rates_df["date"].isin(dates)) & (rates_df["from_currency"] == "USD")]
# Drop unneeded columns
rates_df.drop(columns=['from_currency', 'amount', 'to_currency','updated_at'], inplace=True)
rates_df.rename(columns={'date': 'exchange_rate_date'}, inplace=True)

# Merge exchange rates with product data using a cross join
products_df = store_items.merge(rates_df, how='cross')
products_df["price_eur"] = products_df["price_usd"] * products_df["exchange_rate"]

products_df.head()

,id,title,price_usd,description,category,image,rating_rate,rating_count,exchange_rate_date,exchange_rate,price_eur
0,15,BIYLACLESEN Women's 3-in-1 Snowboard Jacket Wi...,56.99,"Note:The Jackets is US standard size, Please c...",women's clothing,https://fakestoreapi.com/img/51Y5NI-I5jL._AC_U...,2.6,235,2024-02-12,0.92825,52.900968
1,15,BIYLACLESEN Women's 3-in-1 Snowboard Jacket Wi...,56.99,"Note:The Jackets is US standard size, Please c...",women's clothing,https://fakestoreapi.com/img/51Y5NI-I5jL._AC_U...,2.6,235,2024-02-13,0.92653,52.802945
2,16,Lock and Love Women's Removable Hooded Faux Le...,29.95,100% POLYURETHANE(shell) 100% POLYESTER(lining...,women's clothing,https://fakestoreapi.com/img/81XH0e8fefL._AC_U...,2.9,340,2024-02-12,0.92825,27.801088
3,16,Lock and Love Women's Removable Hooded Faux Le...,29.95,100% POLYURETHANE(shell) 100% POLYESTER(lining...,women's clothing,https://fakestoreapi.com/img/81XH0e8fefL._AC_U...,2.9,340,2024-02-13,0.92653,27.749573
4,17,Rain Jacket Women Windbreaker Striped Climbing...,39.99,Lightweight perfet for trip or casual wear---L...,women's clothing,https://fakestoreapi.com/img/71HblAHs5xL._AC_U...,3.8,679,2024-02-12,0.92825,37.120718


In [5]:
# I'm using the DuckDB library to mimic a relational database here.
# Check the "DDL_products_big_query.sql" file for the actual table creation script for BigQuery.

# Create statement for the final table
ddl_query = """
        -- Drop the table if it exists
        DROP TABLE IF EXISTS top_products;

        -- Create the table with appropriate data types
        CREATE TABLE top_products (
            id INTEGER,
            title STRING,
            price_usd FLOAT,
            description STRING,
            category STRING,
            image STRING,
            rating_rate FLOAT,
            rating_count INTEGER,
            exchange_rate_date DATE,
            exchange_rate FLOAT,
            price_eur FLOAT
        );
    """
# I'm assuming that the required Top 5 products should be just five items, otherwise this would be a DENSE_RANK().
# In a real scenario this would be improved, for example, using other fields for cases where there are multiple items with the exact same rank.
top_products_query ="""
        INSERT INTO
            top_products
        SELECT
            id,
            title,
            price_usd,
            description,
            category,
            image,
            rating_rate,
            rating_count,
            exchange_rate_date,
            exchange_rate,
            price_eur
        FROM
            (
                SELECT
                    id,
                    title,
                    price_usd,
                    description,
                    category,
                    image,
                    rating_rate,
                    rating_count,
                    exchange_rate_date,
                    exchange_rate,
                    price_eur,
                    rank() OVER (
                        PARTITION BY exchange_rate_date
                        ORDER BY
                            rating_rate DESC
                    ) AS rank
                FROM
                    all_products
            )
        WHERE
            rank <= 5
    """

# Get the data from the "products_df" DataFrame into DuckDB
duckdb.sql("CREATE OR REPLACE TABLE all_products AS SELECT * FROM products_df")
# Create the "top_products" table
duckdb.sql(ddl_query)
# Insert the top products into the "top_products" table using the "top_products_query" query
duckdb.sql(top_products_query)
#Show the results
duckdb.sql("SELECT * FROM top_products").show()

┌───────┬──────────────────────┬───────────┬───┬──────────────┬────────────────────┬───────────────┬────────────┐
│  id   │        title         │ price_usd │ … │ rating_count │ exchange_rate_date │ exchange_rate │ price_eur  │
│ int32 │       varchar        │   float   │   │    int32     │        date        │     float     │   float    │
├───────┼──────────────────────┼───────────┼───┼──────────────┼────────────────────┼───────────────┼────────────┤
│    18 │ MBJ Women's Solid …  │      9.85 │ … │          130 │ 2024-02-13         │       0.92653 │   9.126321 │
│    19 │ Opna Women's Short…  │      7.95 │ … │          146 │ 2024-02-13         │       0.92653 │  7.3659134 │
│    17 │ Rain Jacket Women …  │     39.99 │ … │          679 │ 2024-02-13         │       0.92653 │  37.051933 │
│    20 │ DANVOUY Womens T S…  │     12.99 │ … │          145 │ 2024-02-13         │       0.92653 │ 12.0356245 │
│    16 │ Lock and Love Wome…  │     29.95 │ … │          340 │ 2024-02-13         │    